In [1]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.impute import SimpleImputer
# import preprocessing from sklearn
from sklearn import preprocessing

In [2]:
# people = pd.read_csv("people_Drehem.csv")
people = pd.read_csv("Graph Representation/people_and_target.csv")
archives = pd.read_csv("archive_map.csv")
archives['PID'] = [int(archives['PID'][i][1:]) for i in np.arange(len(archives['PID']))]
people = pd.merge(people, archives, how='inner', left_on='p index', right_on='PID')
people = people.drop(['Unnamed: 0', 'Unnamed: 0.1','PID', 'normalized name', 'family', 'date name', 'p index', 'Archive_y'], axis=1).rename(index=str, columns={'Archive_x': 'Archive'})
people = people.sort_values(['name'])

/Users/babybear68/anaconda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#For now, I am dropping rows that have missing values, since DBSCAN only works when there are no NANs
# people = people.dropna()

Our people dataframe contains columns for the variables we want to use on our clustering algorithm. Our next step is to convert our categorical data into numerical data. We can do this by creating a binary matrix. 

In [4]:
# people_cat = people.select_dtypes(include=[object])
# people_cat['profession'] = people_cat['profession'].astype(str)
# people_cat['roles'] = people_cat['roles'].astype(str)
# people_cat.head(3)

In [5]:
imp = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
lst = []
for n in people['name'].unique():
    x = people[people['name'] == n]
    flag = False
    for c in x.columns:
        if x[c].isnull().all():
            flag = True
            break
    if flag:
        continue
    lst.append(pd.DataFrame(imp.fit_transform(x)))

In [6]:
len(lst)

203

In [7]:
people_new = pd.DataFrame()
for i in lst:
    people_new = people_new.append(i)

In [8]:
people_new.columns = people.columns

In [9]:
people_new.head()

,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
0,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
1,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
2,...-a,{'source'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
3,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
4,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN


In [10]:
# TODO: create a LabelEncoder object and fit it to each feature in people_cat


# 1. INSTANTIATE
# encode labels with value between 0 and n_classes-1.
le = preprocessing.LabelEncoder()


# 2/3. FIT AND TRANSFORM
# use df.apply() to apply le.fit_transform to all columns
people_cat_2 = people_new.apply(le.fit_transform)
people_cat_2.head()

,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
0,0,8,70,141,1,0,284,5,242,189
1,0,8,70,141,4,0,284,5,242,189
2,0,9,70,141,4,0,284,5,242,189
3,0,8,70,141,4,0,284,5,242,189
4,0,8,70,141,1,0,284,5,242,189


In [11]:
# TODO: create a OneHotEncoder object, and fit it to all of people_cat

# 1. INSTANTIATE
enc = preprocessing.OneHotEncoder()

# 2. FIT
enc.fit(people_cat_2)

# 3. Transform
onehotlabels = enc.transform(people_cat_2).toarray()
onehotlabels.shape

# as you can see, you've the same number of rows 891
# but now you've so many more columns due to how we changed all the categorical data into numerical data

/Users/babybear68/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/_encoders.py:363: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


(38461, 4946)

In [12]:
onehotlabels

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

We just converted our categorical data into a sparse matrix. We should add this matrix to our quantitative data. 

In [13]:
dates = [[people_new['processed date'].values[i] / 30] for i in np.arange(len(people_new['processed date']))]
people_cat_quant = np.hstack((onehotlabels, dates))
people_cat_quant

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.03467   ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.03467   ],
       [1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.03467   ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.20266667],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.46701   ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        2.43374333]])

Our matrix is now ready to be passed through the DBSCAN algorithm. 

Let's try the algorithm with the name "...-a"

In [14]:
people_new[people_new['name'] == 'ur-mes'].head()

,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
0,ur-mes,{'recipient'},ensik[ruler],78.09,domesticated animal,1555,1565,{'intermediary'},ur-mes,Urmes[]PN
1,ur-mes,{'recipient'},ensik[ruler],78.09,unknown,1555,1555,{'intermediary'},ur-mes,Urmes[]PN
2,ur-mes,{'recipient'},ensik[ruler],78.09,domesticated animal,1555,1555,{'intermediary'},ur-mes,Urmes[]PN
3,ur-mes,{'recipient'},ensik[ruler],78.09,unknown,1555,1555,{'intermediary'},ur-mes,Urmes[]PN
4,ur-mes,{'recipient'},ensik[ruler],78.09,domesticated animal,1555,1555,{'intermediary'},ur-mes,Urmes[]PN


In [15]:
people_new.head()

,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
0,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
1,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
2,...-a,{'source'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
3,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
4,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN


In [16]:
# people.loc[people['name'] == "šu-{d}šul-gi"]
name_example = people_cat_quant[people_cat_quant[:,0] == 1,:]

In [17]:
ni = {'name': people.groupby(['name']).size().keys()}
names_and_indices = pd.DataFrame(data=ni)
names_and_indices.head()

,name
0,...-a
1,...-bi
2,...-du
3,...-ga
4,...-gu₇


In [18]:
clustering = DBSCAN(eps=2).fit(name_example)
clustering.labels_

array([ 0,  0,  0,  0,  0,  0, -1,  0, -1, -1])

In [19]:
a = people_new.loc[people_new['name'] == "...-a"]
a.reset_index()[pd.Series(clustering.labels_) == 0]

,index,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
0,0,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
1,1,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
2,2,...-a,{'source'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
3,3,...-a,{'representative'},udu[sheep],61.0401,queen,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
4,4,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
5,5,...-a,{'representative'},udu[sheep],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN
7,7,...-a,{'representative'},nubanda[overseer],61.0401,domesticated animal,1,1189,{'recipient'},lu₂-{d}inanna,LuʾInanakak[]PN


In [20]:
a.reset_index()[pd.Series(clustering.labels_) == -1]

,index,name,roles,profession,processed date,Archive,source,target,target's role,target_name,target norm_name
6,6,...-a,{'representative'},udu[sheep],62.04,domesticated animal,3,884,{'intermediary'},ka₅-a-mu,LuʾInanakak[]PN
8,8,...-a,{'representative'},udu[sheep],61.0401,dead animal,1,2181,{'recipient'},šeš-kal-la,Šeškala[]PN
9,9,...-a,{'representative'},udu[sheep],61.0401,dead animal,1,457,{'recipient'},dingir-ba-ni,Ilumbani[]PN
